## Importing the relevant libraries that will be used in the upcoming codes

In [27]:
import networkx as ntx
import matplotlib.pyplot as plt
from collections import deque

# DFS

## The DFS function

In [ ]:
def dfs_tree(graph, start):
    visited = set()
    stack = [(start, None)]  # (node, parent)

    dfs_tree_edges = {}

    while stack:
        node, parent = stack.pop()

        if node not in visited:
            visited.add(node)

            if parent is not None:
                dfs_tree_edges[(parent, node)] = True

            # Explore neighbors in reverse order to match the order of the dfs_edges function
            neighbors = list(graph.neighbors(node))
            neighbors.reverse()
            stack.extend((neighbor, node) for neighbor in neighbors if neighbor not in visited)

    return dfs_tree_edges


## Running DFS on a directed graph

In [ ]:
# Creating a directed graph
G = ntx.DiGraph()

# Adding nodes and edges to the graph
# Node 4 is intentionally left dead
# Edge going from 6 to 9 is a bridge
G.add_edges_from([(3, 14), (3, 13), (7, 3), (13, 1), (6, 13), (7, 6),
                  (1, 7), (4, 7), (6, 9), (2, 1), (6, 2), (2, 11), (11, 10),
                  (9, 5), (12, 5), (8, 9), (5, 8), (8, 15), (15, 12)])

# Set start node as wished and run DFS function to obtain the edge set for DFS tree
start_node = 1
dfs_tree_edges = dfs_tree(G, start_node)

# Creating the DFS graph with obtained edge set
dfs_tree_graph = ntx.DiGraph(list(dfs_tree_edges.keys()))

# Planar layout used to create a graph that will easily convey information
pos = ntx.planar_layout(G)

# Both the original and DFS versions of the graph are displayed to show the 
# DFS tree in contrast to the original graph it was performed on
ntx.draw(G, pos, with_labels=True, font_weight='bold', node_color='lightblue', edge_color='gray')
ntx.draw(dfs_tree_graph, pos, with_labels=True, font_weight='bold', node_color='salmon', edge_color='red', style='dashed')

plt.show()

## Running DFS on an undirected graph

In [ ]:
G = ntx.Graph()

G.add_edges_from([(3, 14), (3, 13), (7, 3), (13, 1), (6, 13), (7, 6),
                  (1, 7), (4, 7), (6, 9), (2, 1), (6, 2), (2, 11), (11, 10),
                  (9, 5), (12, 5), (8, 9), (5, 8), (8, 15), (15, 12)])

start_node = 1
dfs_tree_edges = dfs_tree(G, start_node)

dfs_tree_graph = ntx.Graph(list(dfs_tree_edges.keys()))

pos = ntx.planar_layout(G)

ntx.draw(G, pos, with_labels=True, font_weight='bold', node_color='lightblue', edge_color='gray')
ntx.draw(dfs_tree_graph, pos, with_labels=True, font_weight='bold', node_color='salmon', edge_color='red', style='dashed')

plt.show()

# BFS

## Function for BFS

In [ ]:
def bfs_tree(graph, start):
    visited = set()
    queue = deque([(start, None)])

    bfs_tree_edges = {}

    while queue:
        node, parent = queue.popleft()

        if node not in visited:
            visited.add(node)

            if parent is not None:
                bfs_tree_edges[(parent, node)] = True

            # Explore neighbors in order
            neighbors = list(graph.neighbors(node))
            queue.extend((neighbor, node) for neighbor in neighbors if neighbor not in visited)

    return bfs_tree_edges

## Running BFS on a directed graph

In [ ]:
# Creating a directed graph
G = ntx.DiGraph()

# Add nodes and edges to the graph
G.add_edges_from([(3, 14), (3, 13), (7, 3), (13, 1), (6, 13), (7, 6),
                  (1, 7), (4, 7), (6, 9), (2, 1), (6, 2), (2, 11), (11, 10),
                  (9, 5), (12, 5), (8, 9), (5, 8), (8, 15), (15, 12)])

# Set start node as wished and run BFS function to obtain the edge set for BFS tree
start_node = 1
bfs_tree_edges = bfs_tree(G, start_node)

# Creating the BFS graph with obtained edge set
bfs_tree_graph = ntx.DiGraph(list(bfs_tree_edges.keys()))

# Planar layout used to create a graph that will easily convey information
pos = ntx.planar_layout(G)

# Both the original and DFS versions of the graph are displayed to show the 
# BFS tree in contrast to the original graph it was performed on
ntx.draw(G, pos, with_labels=True, font_weight='bold', node_color='lightblue', edge_color='gray')
ntx.draw(bfs_tree_graph, pos, with_labels=True, font_weight='bold', node_color='salmon', edge_color='red', style='dashed')

plt.show()


## Running BFS on an undirected graph

In [ ]:
G = ntx.Graph()

G.add_edges_from([(3, 14), (3, 13), (7, 3), (13, 1), (6, 13), (7, 6),
                  (1, 7), (4, 7), (6, 9), (2, 1), (6, 2), (2, 11), (11, 10),
                  (9, 5), (12, 5), (8, 9), (5, 8), (8, 15), (15, 12)])

start_node = 1
bfs_tree_edges = bfs_tree(G, start_node)

bfs_tree_graph = ntx.Graph(list(bfs_tree_edges.keys()))

pos = ntx.planar_layout(G)

ntx.draw(G, pos, with_labels=True, font_weight='bold', node_color='lightblue', edge_color='gray')
ntx.draw(bfs_tree_graph, pos, with_labels=True, font_weight='bold', node_color='salmon', edge_color='red', style='dashed')

plt.show()


# Max-Flow Algorithm

#### Function to check whether a path from source to sink exists (stopping criteria)

In [35]:
def check_valid_path(graph, current_node, sink, current_path):
    if current_node == sink:
        return current_node

    for edge in graph.neighbors(current_node):
        slack = graph[current_node][edge]['capacity'] - graph[current_node][edge]['flow']
        if slack > 0 and edge not in current_path:
            resultant_path = check_valid_path(graph, edge, sink, current_path + [(current_node, edge)])
        if resultant_path is not None:
            return resultant_path

#### Code for the augmenting flow algorithm

In [36]:
def augmenting_flow(graph, source, sink):
    max_flow = 0
    iterated_path = check_valid_path(graph, source, sink, [])

    while iterated_path is not None:
        for u, v in iterated_path:
            slack = graph[u][v]['capacity'] - graph[u][v]['flow']
            min_cap = min(slack, min_cap)
        
        for u, v in iterated_path:
            graph[u][v]['flow'] += min_cap
        
        max_flow += min_cap

        ## Print graph of iterated path here

        iterated_path = check_valid_path(graph, source, sink, [])

    return max_flow

In [ ]:
G = ntx.DiGraph()
G.add_edge(1, 2, capacity = 1, flow = 0)